<a href="https://colab.research.google.com/github/clarariachi/Topic_Modelling_LDA/blob/main/Topic_Modelling_With_NIPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all needed libraries, transfer my workspace to google colab, and load csv files into dataframes
import pandas as pd
import gensim 
import wordcloud
import gensim.downloader as api
!pip install -U scikit-learn
!pip3 install pyLDAvis # for visualizing topic models
import nltk # for preprocessing
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
import os
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)            
WORK_AREA = "/content/gdrive/" + r'MyDrive/neurips_project/'
os.chdir(WORK_AREA) 

authors_url = 'https://drive.google.com/uc?id=1n8TifV2zNsePkVHXv8iOTKObSA_rhfPS'
authors = pd.read_csv(f'{WORK_AREA}/authors_nips.csv')

papers_url = 'https://drive.google.com/uc?id=1BsRS4uD54hupdk7XI4S3o5tIpql5koAH' 
papers = pd.read_csv(f'{WORK_AREA}/papers.csv')

#papers = pd.merge(authors, docs, on = "source_id")

#papers.drop(['source_id'], axis=1, inplace=True) # removing metadata
print(papers.head())

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Mounted at /content/gdrive
   source_id  year                                              title  \
0         27  1987                         Bit-Serial Neural Networks   
1         63  1987                        Connectivity Versus Entropy   
2         60  1987        The Hopfield Model with Multi-Level Neurons   
3         59  1987                               How Neural Nets Work   
4         69  1987  Spatial Organization of Neural Networks: A Pro...   

  abstract                                          full_text  
0      NaN  573 \n\nBIT - SERIAL NEURAL  NETWORKS \n\nAlan...  
1      NaN  1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser  S...  
2      NaN  278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...  
3      NaN  442 \n\nAlan  Lapedes \nRobert  Farber \n\nThe...  
4      NaN  740 \n\nSPATIAL  ORGANIZATION  OF  NEURAL  NEn...  


In [ ]:
# cleaning & preprocessing full_text

# tokenization of full_text
from nltk.tokenize import word_tokenize
papers['preprocessed_text'] = papers['full_text'].apply(lambda x: word_tokenize(str(x))) # splitting each string of text in the full_text column into a list of individual words
papers['preprocessed_text'] = papers['preprocessed_text'].apply(lambda x: [word.lower() for word in x if word.isalpha() and len(word)>3]) # removes punctuation, special characters, and numbers from the paper text data

# retrieving english stopwords corpus from NLTK
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'indeed', 'using', 'given', 'used', 'show', 'even', 'know', 'look', 'page', 'would', 'select', 'choose', 'university', 'example', 'group', 'unit', 'much', 'many', 'form', 'also', 'note', 'case', 'particular', 'could', 'might', 'approximate', 'about', 'thus', 'therefore', 'et', 'al', 'f', 'n', 'x', 'y', 'eg', 'ie', 'p', 'well', 'give', 'word', 'although', 'though', 'either', 'general', 'assume', 'second', 'represent', 'respective', 'correspond', 'input', 'output', 'finally', 'fact', 'define', 'update', 'next', 'compute', 'pair', 'require', 'label', 'change']) # removing more stopwords not already in the NLTK corpus and which are specific to NIPS papers to further reduce noise
stop_words_set = set(stop_words)
papers['preprocessed_text'] = papers['preprocessed_text'].apply(lambda x: [word for word in x if word not in stop_words_set]) # removing stopwords

# lemmatizing words in paper_text
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() # creating an instance of the WordNetLemmatizer class
papers['preprocessed_text'] = papers['preprocessed_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x]) # passing each word in the text of each document into the lemmatize method 

print(papers.head())

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   source_id  year                                              title  \
0         27  1987                         Bit-Serial Neural Networks   
1         63  1987                        Connectivity Versus Entropy   
2         60  1987        The Hopfield Model with Multi-Level Neurons   
3         59  1987                               How Neural Nets Work   
4         69  1987  Spatial Organization of Neural Networks: A Pro...   

  abstract                                          full_text  \
0      NaN  573 \n\nBIT - SERIAL NEURAL  NETWORKS \n\nAlan...   
1      NaN  1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser  S...   
2      NaN  278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...   
3      NaN  442 \n\nAlan  Lapedes \nRobert  Farber \n\nThe...   
4      NaN  740 \n\nSPATIAL  ORGANIZATION  OF  NEURAL  NEn...   

                                   preprocessed_text  
0  [serial, neural, network, alan, murray, anthon...  
1  [connectivity, versus, entropy, yaser, califor...  
2  [h

In [ ]:
# creating a list of sublists where each sublist contains the words of each document
tokenized_papers = [row for row in papers['preprocessed_text']]
print(tokenized_papers[0], tokenized_papers[1], end = "\n")

['serial', 'neural', 'network', 'alan', 'murray', 'anthony', 'smith', 'butler', 'department', 'electrical', 'engineering', 'edinburgh', 'king', 'building', 'mayfield', 'road', 'edinburgh', 'scotland', 'abstract', 'serial', 'vlsi', 'neural', 'network', 'described', 'initial', 'architecture', 'synapse', 'array', 'silicon', 'layout', 'board', 'design', 'issue', 'surrounding', 'serial', 'computation', 'arithmetic', 'discussed', 'parallel', 'development', 'hybrid', 'neural', 'network', 'outlined', 'learning', 'recall', 'capability', 'reported', 'serial', 'network', 'along', 'projected', 'specification', 'neuron', 'serial', 'board', 'operating', 'tech', 'nique', 'extended', 'synapsis', 'network', 'time', 'paging', 'technique', 'time', 'multiplex', 'calculation', 'synapse', 'array', 'introduction', 'function', 'synthetic', 'neural', 'network', 'aspire', 'mimic', 'ability', 'sider', 'solution', 'simultaneously', 'ability', 'work', 'corrupted', 'data', 'natural', 'fault', 'tolerance', 'arises',

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# incorporating bigrams and trigrams e.g. "machine learning" is considered as "machine_learning" if it appears at least 20 times in a document
from gensim.models import Phrases
from gensim.models.phrases import Phraser
n_grams = Phrases(tokenized_papers, min_count = 20)
phraser_model = Phraser(n_grams)
phrased_corpus = [phraser_model[doc] for doc in tokenized_papers]

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# for debugging
print(phrased_corpus[0])
print(phrased_corpus[1])
print(phrased_corpus[2])
print(phrased_corpus[3])

['serial', 'neural_network', 'alan', 'murray', 'anthony', 'smith', 'butler', 'department_electrical', 'engineering', 'edinburgh', 'king', 'building', 'mayfield', 'road', 'edinburgh_scotland', 'abstract', 'serial', 'vlsi', 'neural_network', 'described', 'initial', 'architecture', 'synapse', 'array', 'silicon', 'layout', 'board', 'design', 'issue', 'surrounding', 'serial', 'computation', 'arithmetic', 'discussed', 'parallel', 'development', 'hybrid', 'neural_network', 'outlined', 'learning', 'recall', 'capability', 'reported', 'serial', 'network', 'along', 'projected', 'specification', 'neuron', 'serial', 'board', 'operating', 'tech_nique', 'extended', 'synapsis', 'network', 'time', 'using', 'paging', 'technique', 'time', 'multiplex', 'calculation', 'synapse', 'array', 'introduction', 'function', 'synthetic', 'neural_network', 'aspire', 'mimic', 'ability', 'sider', 'solution', 'simultaneously', 'ability', 'work', 'corrupted', 'data', 'natural', 'fault_tolerance', 'arises', 'parallelism',

In [ ]:
# Create a dictionary that maps tokens (words) to unique integer ids for each word
import gensim.corpora as corpora
dictionary = corpora.Dictionary(phrased_corpus) # this dictionary will be used to create the bag-of-words representation for each paper
# Filter out words that occur too frequently or infrequently in the corpus
dictionary.filter_extremes(no_below=400, no_above=0.8)
# Create the bag-of-words representation for each paper
corpus = [dictionary.doc2bow(doc) for doc in phrased_corpus] # the doc2bow method returns a list of tuples where the 1st element of the tuple is the unique integer id of each word and the 2nd element is the frequency of each word in the document

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# for debugging
print(corpus[0])
print(corpus[1])
print(corpus[2])
print(corpus[3])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 2), (5, 4), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 30), (16, 10), (17, 2), (18, 4), (19, 2), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 2), (32, 3), (33, 3), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 9), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 4), (47, 1), (48, 1), (49, 1), (50, 1), (51, 9), (52, 4), (53, 1), (54, 12), (55, 3), (56, 22), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 6), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 3), (80, 2), (81, 2), (82, 1), (83, 1), (84, 4), (85, 2), (86, 13), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 8), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110

In [ ]:
# Split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
# before bow
training_docs, testing_docs = train_test_split(phrased_corpus, test_size = 0.2, random_state = 42)
# after bow
training_papers, testing_papers = train_test_split(corpus, test_size=0.2, random_state=42) # set random_state parameter to an arbitrary fixed value (42) to ensure 80-20 split each time we run the code

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from pprint import pprint
# set the desired number of topics (hyperparameter)
num_topics = 10
# Build LDA model
temp = dictionary[0] # loads the dictionary to be able to use it
id2word=dictionary.id2token # creates a dictionary that maps the integer id back to the corresponding word (token)
lda_model = gensim.models.LdaMulticore(corpus=training_papers, id2word=id2word, chunksize=8000,
    num_topics=num_topics,
    passes=10,) # LdaMulticore is used instead of LdaModel for large corpuses of text data. 
# Prints the keywords in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
# Prints the most dominant topic in each document
for i, doc in enumerate(doc_lda):
    print('Document', i)
    dominant_topic = max(doc, key=lambda x: x[1])[0]
    print('Dominant topic:', dominant_topic)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
Document 7180
Dominant topic: 2
Document 7181
Dominant topic: 1
Document 7182
Dominant topic: 2
Document 7183
Dominant topic: 3
Document 7184
Dominant topic: 2
Document 7185
Dominant topic: 0
Document 7186
Dominant topic: 9
Document 7187
Dominant topic: 2
Document 7188
Dominant topic: 3
Document 7189
Dominant topic: 3
Document 7190
Dominant topic: 3
Document 7191
Dominant topic: 5
Document 7192
Dominant topic: 9
Document 7193
Dominant topic: 1
Document 7194
Dominant topic: 6
Document 7195
Dominant topic: 5
Document 7196
Dominant topic: 1
Document 7197
Dominant topic: 6
Document 7198
Dominant topic: 5
Document 7199
Dominant topic: 0
Document 7200
Dominant topic: 2
Document 7201
Dominant topic: 5
Document 7202
Dominant topic: 9
Document 7203
Dominant topic: 3
Document 7204
Dominant topic: 2
Document 7205
Dominant topic: 6
Document 7206
Dominant topic: 5
Document 7207
Dominant topic: 2
Document 7208
Dominant topic: 3
Document 7209
Dominan

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"graph" + 0.019*"node" + 0.014*"matrix" + 0.010*"variable" + '
  '0.010*"edge" + 0.008*"network" + 0.008*"structure" + 0.006*"cluster" + '
  '0.005*"clustering" + 0.005*"size"'),
 (1,
  '0.012*"loss" + 0.010*"class" + 0.009*"training" + 0.009*"bound" + '
  '0.008*"error" + 0.008*"point" + 0.008*"distribution" + 0.008*"sample" + '
  '0.007*"theorem" + 0.006*"classiﬁcation"'),
 (2,
  '0.010*"theorem" + 0.009*"bound" + 0.008*"gradient" + 0.007*"optimization" + '
  '0.006*"convergence" + 0.006*"analysis" + 0.006*"iteration" + '
  '0.006*"setting" + 0.006*"rate" + 0.006*"matrix"'),
 (3,
  '0.014*"distribution" + 0.014*"kernel" + 0.011*"sample" + 0.009*"matrix" + '
  '0.007*"point" + 0.007*"estimate" + 0.007*"prior" + 0.006*"posterior" + '
  '0.006*"gaussian" + 0.005*"test"'),
 (4,
  '0.015*"neuron" + 0.010*"network" + 0.008*"response" + 0.008*"system" + '
  '0.008*"signal" + 0.007*"stimulus" + 0.007*"neural" + 0.006*"cell" + '
  '0.005*"pattern" + 0.005*"dynamic"'),
 (5,
  '0.

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
vis = gensimvis.prepare(lda_model, training_papers, dictionary=dictionary)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [ ]:
from gensim.models import CoherenceModel
d = corpora.Dictionary()
d.id2token = id2word
d.token2id = {v: k for k, v in id2word.items()}
coherence_model_lda = CoherenceModel(model=lda_model, texts=training_docs, dictionary = d, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score for Training Set: ', coherence_lda)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score for Training Set:  0.4199226294096129


In [ ]:
test_topic_distribution = []
for doc_bow in testing_papers:
    topic_distribution = lda_model.get_document_topics(doc_bow)
    test_topic_distribution.append(topic_distribution)

coherence_model_lda_test = CoherenceModel(model=lda_model, texts=testing_docs, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model_lda_test.get_coherence()

print('\nCoherence Score for Test Set: ', coherence_score)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score for Test Set:  0.42057043225013996


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Visualize the topics
vis = gensimvis.prepare(lda_model, testing_papers, dictionary=d)
pyLDAvis.display(vis)

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
